In [1]:
# Import library
import pandas as pd
import numpy as np
import os, random, warnings, gc, psutil, datetime
from tqdm import tqdm_notebook, tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

import lightgbm as lgbm
from catboost import CatBoostRegressor

from glob import glob
from IPython.display import display

import seaborn as sns
import matplotlib.pyplot as plt

# Set options
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)
pd.options.display.max_colwidth = 300

warnings.filterwarnings('ignore')

%matplotlib inline
sns.set_palette('bright')

In [2]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything()

In [3]:
train = pd.read_pickle('../Create_Features/preprocessed_train.pickle')
test = pd.read_pickle('../Create_Features/preprocessed_test.pickle')

In [4]:
n_trn = 415423
target_col = '18~20_ride'

In [5]:
train.head()

,18~20_ride,bus_route_id,in_out,latitude,longitude,station_code,dayofweek,weekend,ride_total,takeoff_total,ride_go_to_work,takeoff_go_to_work,dis_jejusi,dis_seoquipo,bus_route_id_station_code,bus_route_id_station_code_weekend,date_fq_enc,station_code_fq_enc,bus_route_id_fq_enc,bus_route_id_station_code_fq_enc,date_bus_route_id_fq_enc,date_station_code_fq_enc,date_bus_route_id_station_code_fq_enc,7~8_ride_date_mean,7~8_ride_date_bus_route_id_mean,8~9_ride_date_mean,8~9_ride_date_bus_route_id_mean,9~10_ride_date_mean,9~10_ride_date_bus_route_id_mean,station_sequence,station_reverse_sequence,weekday,is_national_holiday,getin_total,morning_getin,morning_takeoff,noon_getin,noon_takeoff,station_morning_getin_sum,station_morning_takeoff_sum,bus_route_getin_sum,bus_route_takeoff_sum,station_morning_getin_mean,station_morning_takeoff_mean,bus_route_getin_mean,bus_route_takeoff_mean,kmeans1,kmeans2,regular_commuter_count,afternoon_takeoff,next_bus_time_diff,getin_user_count1_morning,getin_user_count2_morning,takeoff_user_count1_noon,takeoff_user_count2_noon,hourly_rain,prev_daily_rain,hourly_cloud,latlong_second,total_population,man_population,woman_population,avg_time_diff,passengers_in,passengers_out,latitude_rank,longitude_rank
0,0.0,0,1,33.48990,126.49373,322,6,1,16.0,0.0,8.0,0.0,2.954920,26.256744,31053,53745,11538,46,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,1,25,6,0,16.0,3.0,0.0,13.0,0.0,3.0,0.0,50.0,8.0,3.000000,0.000000,2.0,0.32,15,9,9.0,0.0,3650.490741,7.0,1.0,NaN,NaN,0.2,0.0,88,2411,43217.0,21189.0,22028.0,3113.015748,15.239130,1.909091,404772.5,214964.5
1,5.0,0,1,33.48944,126.48508,335,6,1,22.0,0.0,10.0,0.0,3.720275,26.403025,31054,53747,11538,2303,1189,46,25,45,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,2,24,6,0,22.0,9.0,0.0,13.0,0.0,183.0,116.0,50.0,8.0,4.066667,2.577778,2.0,0.32,15,9,466.0,0.0,3650.490741,197.0,NaN,95.0,NaN,0.2,0.0,88,2403,43217.0,21189.0,22028.0,1837.940774,594.000000,428.000000,396531.5,189710.0
2,2.0,0,1,33.48181,126.47352,408,6,1,4.0,0.0,3.0,0.0,5.036124,25.893305,31057,53753,11538,1154,1189,46,25,21,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,3,23,6,0,4.0,2.0,0.0,2.0,0.0,71.0,11.0,50.0,8.0,3.380952,0.523810,2.0,0.32,15,45,164.0,0.0,3650.490741,76.0,NaN,11.0,2.0,0.2,0.0,88,2347,56223.0,27761.0,28462.0,2448.248012,166.913043,28.000000,342909.5,156930.5
3,53.0,0,0,33.50577,126.49252,1448,6,1,79.0,0.0,49.0,0.0,2.864166,27.997494,31020,53682,11538,49,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,4,22,6,0,79.0,23.0,0.0,56.0,0.0,23.0,0.0,50.0,8.0,23.000000,0.000000,2.0,0.32,4,3,2.0,0.0,3650.490741,49.0,NaN,NaN,NaN,0.2,0.0,88,2980,15673.0,7904.0,7769.0,3961.540412,89.130435,3.600000,518315.0,210660.0
4,0.0,0,0,33.25579,126.41260,1510,6,1,0.0,1.0,0.0,1.0,29.040353,13.574693,31022,53686,11538,386,1189,39,25,10,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,5,21,6,0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,50.0,8.0,0.100000,0.500000,2.0,0.32,85,136,3.0,0.0,3650.490741,4.0,NaN,5.0,NaN,0.2,0.0,88,562,4414.0,2252.0,2162.0,1058.012605,6.847826,27.456522,98750.5,93507.5


In [7]:
drop_cols = ['hourly_rain',
       'prev_daily_rain', 'hourly_cloud', 'latlong_second', 'total_population',
       'man_population', 'woman_population']

train= train.drop(drop_cols,1)
test= test.drop(drop_cols,1)


In [8]:
train.head()

,18~20_ride,bus_route_id,in_out,latitude,longitude,station_code,dayofweek,weekend,ride_total,takeoff_total,ride_go_to_work,takeoff_go_to_work,dis_jejusi,dis_seoquipo,bus_route_id_station_code,bus_route_id_station_code_weekend,date_fq_enc,station_code_fq_enc,bus_route_id_fq_enc,bus_route_id_station_code_fq_enc,date_bus_route_id_fq_enc,date_station_code_fq_enc,date_bus_route_id_station_code_fq_enc,7~8_ride_date_mean,7~8_ride_date_bus_route_id_mean,8~9_ride_date_mean,8~9_ride_date_bus_route_id_mean,9~10_ride_date_mean,9~10_ride_date_bus_route_id_mean,station_sequence,station_reverse_sequence,weekday,is_national_holiday,getin_total,morning_getin,morning_takeoff,noon_getin,noon_takeoff,station_morning_getin_sum,station_morning_takeoff_sum,bus_route_getin_sum,bus_route_takeoff_sum,station_morning_getin_mean,station_morning_takeoff_mean,bus_route_getin_mean,bus_route_takeoff_mean,kmeans1,kmeans2,regular_commuter_count,afternoon_takeoff,next_bus_time_diff,getin_user_count1_morning,getin_user_count2_morning,takeoff_user_count1_noon,takeoff_user_count2_noon,avg_time_diff,passengers_in,passengers_out,latitude_rank,longitude_rank
0,0.0,0,1,33.48990,126.49373,322,6,1,16.0,0.0,8.0,0.0,2.954920,26.256744,31053,53745,11538,46,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,1,25,6,0,16.0,3.0,0.0,13.0,0.0,3.0,0.0,50.0,8.0,3.000000,0.000000,2.0,0.32,15,9,9.0,0.0,3650.490741,7.0,1.0,NaN,NaN,3113.015748,15.239130,1.909091,404772.5,214964.5
1,5.0,0,1,33.48944,126.48508,335,6,1,22.0,0.0,10.0,0.0,3.720275,26.403025,31054,53747,11538,2303,1189,46,25,45,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,2,24,6,0,22.0,9.0,0.0,13.0,0.0,183.0,116.0,50.0,8.0,4.066667,2.577778,2.0,0.32,15,9,466.0,0.0,3650.490741,197.0,NaN,95.0,NaN,1837.940774,594.000000,428.000000,396531.5,189710.0
2,2.0,0,1,33.48181,126.47352,408,6,1,4.0,0.0,3.0,0.0,5.036124,25.893305,31057,53753,11538,1154,1189,46,25,21,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,3,23,6,0,4.0,2.0,0.0,2.0,0.0,71.0,11.0,50.0,8.0,3.380952,0.523810,2.0,0.32,15,45,164.0,0.0,3650.490741,76.0,NaN,11.0,2.0,2448.248012,166.913043,28.000000,342909.5,156930.5
3,53.0,0,0,33.50577,126.49252,1448,6,1,79.0,0.0,49.0,0.0,2.864166,27.997494,31020,53682,11538,49,1189,46,25,1,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,4,22,6,0,79.0,23.0,0.0,56.0,0.0,23.0,0.0,50.0,8.0,23.000000,0.000000,2.0,0.32,4,3,2.0,0.0,3650.490741,49.0,NaN,NaN,NaN,3961.540412,89.130435,3.600000,518315.0,210660.0
4,0.0,0,0,33.25579,126.41260,1510,6,1,0.0,1.0,0.0,1.0,29.040353,13.574693,31022,53686,11538,386,1189,39,25,10,1,0.391576,1.12,0.49246,0.72,0.543855,1.52,5,21,6,0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,50.0,8.0,0.100000,0.500000,2.0,0.32,85,136,3.0,0.0,3650.490741,4.0,NaN,5.0,NaN,1058.012605,6.847826,27.456522,98750.5,93507.5


In [9]:
# Before modeling
train_set = train.drop(target_col,1)
test_set = test.drop(target_col,1)

train_label = train[target_col]
test_label = test[target_col]

In [11]:
# Basic LGBM Model
n_splits= 5
NUM_BOOST_ROUND = 100000
SEED = 1993
lgbm_param = {'objective':'rmse',
              'boosting_type': 'gbdt',
              'random_state':1993,
              'learning_rate':0.05,
              'subsample':0.7,
              'tree_learner': 'serial',
              'colsample_bytree':0.78,
              'early_stopping_rounds':50,
              'subsample_freq': 1,
              'reg_lambda':7,
              'reg_alpha': 5,
              'num_leaves': 96,
              'seed' : SEED
            }

In [12]:
# StractifiedKfold & bus_route_id
split_col = 'bus_route_id'
len_seeds = 1

outer_stractified_busroute_oof_train = np.zeros( train.shape[0] )
outer_stractified_busroute_oof_test = np.zeros( test.shape[0] )

for _ in tqdm_notebook(range(len_seeds)):
    
    seed = random.randint(1, 100000)
    
    cv_list = []

    oof_train = np.zeros( train.shape[0] )
    final_test = np.zeros( test.shape[0] )

    kfolds = StratifiedKFold(n_splits = n_splits, shuffle=True, random_state=seed )

    for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(train_set, train_set[split_col])) ):

        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
        
        dtrain = lgbm.Dataset( X_train, y_train )
        dvalid = lgbm.Dataset( X_valid, y_valid ,reference=dtrain)
        
        lgbm_param['seed'] = seed

        model = lgbm.train(lgbm_param , dtrain, NUM_BOOST_ROUND, valid_sets=(dtrain, dvalid), valid_names=('train','valid'), 
                            categorical_feature=['bus_route_id','station_code','weekday',\
                                                                'kmeans1','kmeans2',
                                                                ] ,
                           verbose_eval= 100)

        valid_pred = model.predict(X_valid)
        test_pred  = model.predict(test_set)

        oof_train[val_ind] += valid_pred
        final_test += test_pred

        cv_list.append( sqrt(mean_squared_error(y_valid, valid_pred)) )

        print('='*80)

    final_test /= n_splits

    print(f"Average CV : {np.mean(cv_list)}")
    print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, oof_train))}")
    
    outer_stractified_busroute_oof_train += oof_train
    outer_stractified_busroute_oof_test += final_test
    
outer_stractified_busroute_oof_train /=len_seeds
outer_stractified_busroute_oof_test /= len_seeds

print(f"Overall for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

# PostProcessing
outer_stractified_busroute_oof_train = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_train]
outer_stractified_busroute_oof_test = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_test]


print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 1.79989	valid's rmse: 2.23138
[200]	train's rmse: 1.58099	valid's rmse: 2.21109
[300]	train's rmse: 1.4549	valid's rmse: 2.20376
[400]	train's rmse: 1.36994	valid's rmse: 2.19904
[500]	train's rmse: 1.30885	valid's rmse: 2.19551
Early stopping, best iteration is:
[502]	train's rmse: 1.30783	valid's rmse: 2.19534
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 1.7918	valid's rmse: 2.23322
[200]	train's rmse: 1.57235	valid's rmse: 2.222
Early stopping, best iteration is:
[182]	train's rmse: 1.60228	valid's rmse: 2.21904
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 1.78716	valid's rmse: 2.23964
[200]	train's rmse: 1.56736	valid's rmse: 2.19902
Early stopping, best iteration is:
[210]	train's rmse: 1.55149	valid's rmse: 2.19767
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 1.79949	valid's rmse: 2.23365
[

In [13]:
df_sub = pd.read_csv('../raw_dataset/submission_sample.csv')
df_sub['18~20_ride'] = outer_stractified_busroute_oof_test

df_sub.to_csv('../submission/after.csv',index=False)

In [8]:
# StractifiedKfold & bus_route_id
split_col = 'bus_route_id'
len_seeds = 40

outer_stractified_busroute_oof_train = np.zeros( train.shape[0] )
outer_stractified_busroute_oof_test = np.zeros( test.shape[0] )

for _ in tqdm_notebook(range(len_seeds)):
    
    seed = random.randint(1, 100000)
    
    cv_list = []

    oof_train = np.zeros( train.shape[0] )
    final_test = np.zeros( test.shape[0] )

    kfolds = StratifiedKFold(n_splits = n_splits, shuffle=True, random_state=seed )

    for ind, (trn_ind, val_ind) in tqdm_notebook( enumerate(kfolds.split(train_set, train_set[split_col])) ):

        X_train, y_train = train_set.iloc[trn_ind], train_label[trn_ind]
        X_valid, y_valid = train_set.iloc[val_ind], train_label[val_ind]
        
        dtrain = lgbm.Dataset( X_train, y_train )
        dvalid = lgbm.Dataset( X_valid, y_valid ,reference=dtrain)
        
        lgbm_param['seed'] = seed

        model = lgbm.train(lgbm_param , dtrain, NUM_BOOST_ROUND, valid_sets=(dtrain, dvalid), valid_names=('train','valid'), 
                            categorical_feature=['bus_route_id','station_code','weekday',\
                                                                'kmeans1','kmeans2',
                                                                ] ,
                           verbose_eval= 100)

        valid_pred = model.predict(X_valid)
        test_pred  = model.predict(test_set)

        oof_train[val_ind] += valid_pred
        final_test += test_pred

        cv_list.append( sqrt(mean_squared_error(y_valid, valid_pred)) )

        print('='*80)

    final_test /= n_splits

    print(f"Average CV : {np.mean(cv_list)}")
    print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, oof_train))}")
    
    outer_stractified_busroute_oof_train += oof_train
    outer_stractified_busroute_oof_test += final_test
    
outer_stractified_busroute_oof_train /=len_seeds
outer_stractified_busroute_oof_test /= len_seeds

print(f"Overall for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

# PostProcessing
outer_stractified_busroute_oof_train = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_train]
outer_stractified_busroute_oof_test = [x if x>0 else 0 for x in  outer_stractified_busroute_oof_test]


print(f"RMSE for OOF: {sqrt(mean_squared_error(train_label, outer_stractified_busroute_oof_train))}")

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.8298	valid's rmse: 2.88314
[200]	train's rmse: 2.21454	valid's rmse: 2.39326
[300]	train's rmse: 1.98082	valid's rmse: 2.26099
[400]	train's rmse: 1.86106	valid's rmse: 2.21927
[500]	train's rmse: 1.7823	valid's rmse: 2.20237
[600]	train's rmse: 1.72252	valid's rmse: 2.19462
[700]	train's rmse: 1.6724	valid's rmse: 2.18902
[800]	train's rmse: 1.62999	valid's rmse: 2.18555
[900]	train's rmse: 1.59188	valid's rmse: 2.18174
[1000]	train's rmse: 1.55764	valid's rmse: 2.17942
[1100]	train's rmse: 1.52761	valid's rmse: 2.17752
[1200]	train's rmse: 1.49939	valid's rmse: 2.17753
Early stopping, best iteration is:
[1156]	train's rmse: 1.51133	valid's rmse: 2.17674
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.81196	valid's rmse: 2.92512
[200]	train's rmse: 2.1939	valid's rmse: 2.40136
[300]	train's rmse: 1.96161	valid's rmse: 2.26749
[400]	train's rmse: 1.84457	valid's rmse:

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.84162	valid's rmse: 2.82064
[200]	train's rmse: 2.22164	valid's rmse: 2.308
[300]	train's rmse: 1.98959	valid's rmse: 2.16515
[400]	train's rmse: 1.8701	valid's rmse: 2.11841
[500]	train's rmse: 1.79263	valid's rmse: 2.09926
[600]	train's rmse: 1.73213	valid's rmse: 2.08987
[700]	train's rmse: 1.68241	valid's rmse: 2.08577
[800]	train's rmse: 1.63784	valid's rmse: 2.08134
[900]	train's rmse: 1.59949	valid's rmse: 2.07861
Early stopping, best iteration is:
[916]	train's rmse: 1.59357	valid's rmse: 2.07841
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.82405	valid's rmse: 2.975
[200]	train's rmse: 2.20955	valid's rmse: 2.44334
[300]	train's rmse: 1.97884	valid's rmse: 2.27837
[400]	train's rmse: 1.85971	valid's rmse: 2.21464
[500]	train's rmse: 1.78052	valid's rmse: 2.1861
[600]	train's rmse: 1.72061	valid's rmse: 2.17026
[700]	train's rmse: 1.66947	valid's rmse: 2.162

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.83769	valid's rmse: 2.91395
[200]	train's rmse: 2.22047	valid's rmse: 2.39122
[300]	train's rmse: 1.98474	valid's rmse: 2.22671
[400]	train's rmse: 1.86611	valid's rmse: 2.16613
[500]	train's rmse: 1.78723	valid's rmse: 2.14002
[600]	train's rmse: 1.72647	valid's rmse: 2.12734
[700]	train's rmse: 1.67561	valid's rmse: 2.12173
[800]	train's rmse: 1.63165	valid's rmse: 2.11692
[900]	train's rmse: 1.59386	valid's rmse: 2.11359
[1000]	train's rmse: 1.55976	valid's rmse: 2.11096
[1100]	train's rmse: 1.52971	valid's rmse: 2.10973
[1200]	train's rmse: 1.50225	valid's rmse: 2.10917
[1300]	train's rmse: 1.47639	valid's rmse: 2.10881
Early stopping, best iteration is:
[1260]	train's rmse: 1.48637	valid's rmse: 2.10844
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.81075	valid's rmse: 2.99789
[200]	train's rmse: 2.2016	valid's rmse: 2.47244
[300]	train's rmse: 1.96971	valid's r

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.81162	valid's rmse: 2.99487
[200]	train's rmse: 2.19925	valid's rmse: 2.48913
[300]	train's rmse: 1.96603	valid's rmse: 2.33587
[400]	train's rmse: 1.84951	valid's rmse: 2.2739
[500]	train's rmse: 1.77358	valid's rmse: 2.24628
[600]	train's rmse: 1.71447	valid's rmse: 2.22825
[700]	train's rmse: 1.66498	valid's rmse: 2.21613
[800]	train's rmse: 1.62156	valid's rmse: 2.20842
[900]	train's rmse: 1.58373	valid's rmse: 2.20169
[1000]	train's rmse: 1.55002	valid's rmse: 2.19844
[1100]	train's rmse: 1.51987	valid's rmse: 2.19578
[1200]	train's rmse: 1.49239	valid's rmse: 2.1923
Early stopping, best iteration is:
[1231]	train's rmse: 1.48421	valid's rmse: 2.19125
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.82936	valid's rmse: 2.83645
[200]	train's rmse: 2.21167	valid's rmse: 2.36681
[300]	train's rmse: 1.97884	valid's rmse: 2.23465
[400]	train's rmse: 1.85912	valid's rms

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.81158	valid's rmse: 2.99863
[200]	train's rmse: 2.20514	valid's rmse: 2.44126
[300]	train's rmse: 1.9752	valid's rmse: 2.27463
[400]	train's rmse: 1.85637	valid's rmse: 2.20635
[500]	train's rmse: 1.77798	valid's rmse: 2.17384
[600]	train's rmse: 1.71764	valid's rmse: 2.15679
[700]	train's rmse: 1.66798	valid's rmse: 2.14491
[800]	train's rmse: 1.62438	valid's rmse: 2.13937
[900]	train's rmse: 1.58598	valid's rmse: 2.13579
[1000]	train's rmse: 1.55184	valid's rmse: 2.13463
[1100]	train's rmse: 1.52079	valid's rmse: 2.13187
Early stopping, best iteration is:
[1089]	train's rmse: 1.52416	valid's rmse: 2.1317
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.83024	valid's rmse: 2.85482
[200]	train's rmse: 2.20956	valid's rmse: 2.34119
[300]	train's rmse: 1.97318	valid's rmse: 2.20761
[400]	train's rmse: 1.85324	valid's rmse: 2.1689
[500]	train's rmse: 1.7748	valid's rmse: 

...

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.83268	valid's rmse: 2.78742
[200]	train's rmse: 2.2104	valid's rmse: 2.31755
[300]	train's rmse: 1.9726	valid's rmse: 2.21554
[400]	train's rmse: 1.84989	valid's rmse: 2.20045
Early stopping, best iteration is:
[423]	train's rmse: 1.82982	valid's rmse: 2.19865
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.84675	valid's rmse: 2.77916
[200]	train's rmse: 2.223	valid's rmse: 2.29486
[300]	train's rmse: 1.99038	valid's rmse: 2.16629
[400]	train's rmse: 1.87113	valid's rmse: 2.12275
[500]	train's rmse: 1.79248	valid's rmse: 2.10525
[600]	train's rmse: 1.73212	valid's rmse: 2.09379
[700]	train's rmse: 1.68071	valid's rmse: 2.08667
[800]	train's rmse: 1.63772	valid's rmse: 2.08312
[900]	train's rmse: 1.59907	valid's rmse: 2.07826
[1000]	train's rmse: 1.56467	valid's rmse: 2.07611
[1100]	train's rmse: 1.53401	valid's rmse: 2.07474
[1200]	train's rmse: 1.50605	valid's rmse: 

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.8054	valid's rmse: 2.9926
[200]	train's rmse: 2.20334	valid's rmse: 2.45682
[300]	train's rmse: 1.97453	valid's rmse: 2.29091
[400]	train's rmse: 1.85839	valid's rmse: 2.23484
[500]	train's rmse: 1.78047	valid's rmse: 2.20929
[600]	train's rmse: 1.72077	valid's rmse: 2.19583
[700]	train's rmse: 1.67146	valid's rmse: 2.18896
[800]	train's rmse: 1.62885	valid's rmse: 2.18518
[900]	train's rmse: 1.59125	valid's rmse: 2.18233
[1000]	train's rmse: 1.55732	valid's rmse: 2.1799
[1100]	train's rmse: 1.52702	valid's rmse: 2.17919
Early stopping, best iteration is:
[1087]	train's rmse: 1.53069	valid's rmse: 2.17894
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.83525	valid's rmse: 2.84828
[200]	train's rmse: 2.20852	valid's rmse: 2.36303
[300]	train's rmse: 1.96761	valid's rmse: 2.25632
[400]	train's rmse: 1.84605	valid's rmse: 2.23236
[500]	train's rmse: 1.76582	valid's rmse:

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.82702	valid's rmse: 2.88774
[200]	train's rmse: 2.20515	valid's rmse: 2.40179
[300]	train's rmse: 1.96841	valid's rmse: 2.27667
[400]	train's rmse: 1.85098	valid's rmse: 2.24485
[500]	train's rmse: 1.77324	valid's rmse: 2.23622
[600]	train's rmse: 1.71319	valid's rmse: 2.23149
[700]	train's rmse: 1.66339	valid's rmse: 2.23119
Early stopping, best iteration is:
[682]	train's rmse: 1.67195	valid's rmse: 2.23084
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.81702	valid's rmse: 2.9462
[200]	train's rmse: 2.20293	valid's rmse: 2.42997
[300]	train's rmse: 1.97384	valid's rmse: 2.28011
[400]	train's rmse: 1.85451	valid's rmse: 2.22579
[500]	train's rmse: 1.77637	valid's rmse: 2.20254
[600]	train's rmse: 1.71618	valid's rmse: 2.18875
[700]	train's rmse: 1.66669	valid's rmse: 2.18064
[800]	train's rmse: 1.62315	valid's rmse: 2.1755
[900]	train's rmse: 1.58531	valid's rmse: 2

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.83925	valid's rmse: 2.83718
[200]	train's rmse: 2.22348	valid's rmse: 2.32708
[300]	train's rmse: 1.98926	valid's rmse: 2.18167
[400]	train's rmse: 1.8709	valid's rmse: 2.13397
[500]	train's rmse: 1.79213	valid's rmse: 2.11078
[600]	train's rmse: 1.73001	valid's rmse: 2.09731
[700]	train's rmse: 1.6791	valid's rmse: 2.08989
[800]	train's rmse: 1.63552	valid's rmse: 2.0834
[900]	train's rmse: 1.5968	valid's rmse: 2.08043
[1000]	train's rmse: 1.56236	valid's rmse: 2.07868
[1100]	train's rmse: 1.53149	valid's rmse: 2.07635
[1200]	train's rmse: 1.50343	valid's rmse: 2.07541
Early stopping, best iteration is:
[1219]	train's rmse: 1.49831	valid's rmse: 2.07477
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.85894	valid's rmse: 2.73785
[200]	train's rmse: 2.2366	valid's rmse: 2.25407
[300]	train's rmse: 1.99932	valid's rmse: 2.12271
[400]	train's rmse: 1.87733	valid's rmse: 

Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.86449	valid's rmse: 2.66621
[200]	train's rmse: 2.23772	valid's rmse: 2.18032
[300]	train's rmse: 2.00104	valid's rmse: 2.06293
[400]	train's rmse: 1.8805	valid's rmse: 2.03443
[500]	train's rmse: 1.79989	valid's rmse: 2.02397
[600]	train's rmse: 1.7382	valid's rmse: 2.02109
[700]	train's rmse: 1.68706	valid's rmse: 2.01898
[800]	train's rmse: 1.64251	valid's rmse: 2.01702
Early stopping, best iteration is:
[844]	train's rmse: 1.62456	valid's rmse: 2.01645
Training until validation scores don't improve for 50 rounds.
[100]	train's rmse: 2.80049	valid's rmse: 3.05187
[200]	train's rmse: 2.19376	valid's rmse: 2.49474
[300]	train's rmse: 1.96549	valid's rmse: 2.32431
[400]	train's rmse: 1.84924	valid's rmse: 2.26512
[500]	train's rmse: 1.77216	valid's rmse: 2.24049
[600]	train's rmse: 1.71298	valid's rmse: 2.2289
[700]	train's rmse: 1.66353	valid's rmse: 2.2205
[800]	train's rmse: 1.62097	valid's rmse: 2.2

In [9]:
df_oof = pd.read_csv('../raw_dataset/train.csv', usecols = ['id','18~20_ride'])
df_oof['18~20_ride'] = outer_stractified_busroute_oof_train

df_oof.to_csv('../oof/lgbm_40_seeds_stractified5k_bus_route_id.csv',index=False)

df_sub = pd.read_csv('../raw_dataset/submission_sample.csv')
df_sub['18~20_ride'] = outer_stractified_busroute_oof_test

df_sub.to_csv('../submission/lgbm_40_seeds_stractified5k_bus_route_id.csv',index=False)